In [ ]:
import numpy as np
import numpy.random as npr
import pandas as pd
import plotly.graph_objects as go
import scipy.stats as sps
from plotly.subplots import make_subplots
from datetime import datetime

cve = pd.read_csv('../input/cve-common-vulnerabilities-and-exposures/cve.csv', header=0, index_col=0)
products = pd.read_csv('../input/cve-common-vulnerabilities-and-exposures/products.csv', header=0, index_col=0)
vendors = pd.read_csv('../input/cve-common-vulnerabilities-and-exposures/vendors.csv', header=0, index_col=0)

cve.pub_date = pd.to_datetime(cve.pub_date)
npr.seed(42)

In [ ]:
vc = products.vulnerable_product.value_counts()
vc = vc[vc.values >= 256]

def get_ccf(p):
    times = products[products.vulnerable_product == p].join(cve.pub_date).pub_date.sort_values().dropna()
    delay = (times.to_numpy()[1:] - times.to_numpy()[:-1]).astype('timedelta64[D]').astype('float')
    return np.corrcoef(pd.to_numeric(times[:-1]), delay)[0,1]

ccf = pd.Series(vc.index.map(get_ccf), index=vc.index).sort_values()

# 0. Overview

Time series data provides an ordering for list entries. This ordering allows us to analyze features as an ordered 1-D array. Histograms are a data summary that breaks one dimensional data into K distinct buckets. By breaking up data according to time, we can identify key points in time where a variable underwent large scale, sustained change.

NOTE: Histograms, as defined here, can (and usually do) have variable width buckets

# 1. A Naive Algorithm, Dynamic Programming

V-optimality is a condition that states that the optimal K-bucket histogram has the minimum cumulative weighted variance across all K buckets. A O(n²k) dynamic programming solution first precomputes the variance of all possible intervals.

In [ ]:
def v_optimal_dp(X, K=2):
    # Precompute subinterval variance
    N = len(X)
    cost = [[np.std(X[i:j]) for i in range(j)] for j in range(1,N+1)]
    
    # Naive-DP solution in O(n²k) time
    d = np.full((len(X), K+1), np.inf)
    src = np.full((len(X), K+1), -1)
    d[0][0] = 0.
    for k in range(K+1):
        for j in range(N):
            for i in range(j):
                d[j][k] = min(d[j][k], d[i][k-1] + cost[j][i+1])
                if d[j][k] == d[i][k-1] + cost[j][i+1]:
                    src[j][k] = i
                    
    # Backtracking
    cur = N - 1
    bp = [0,N]
    for k in range(K,1,-1):
        bp.append(src[cur][k])
        cur = src[cur][k]
        
    return sorted(bp)

In [ ]:
def variable_width_distplot_factory(X, cats, bins, text, y=[]):
    # X - the data
    # cats - a list of pd.Interval where 
    #       left is the index of the left data point
    #       and right is the index of the right data point
    fig = make_subplots(rows=2, cols=1, shared_xaxes=True, row_heights=[600, 100], vertical_spacing=0)
    N = len(X)
    
    for i, cat in enumerate(cats):
        l = X[min(cat.right, N-1)] - X[min(cat.left, N-1)]
        x = [X[min(cat.left, N-1)] + l/2]
        fig.add_trace(go.Bar(
                x=x,
                y=[len(X[bins == cat])] if len(y) == 0 else [y[i]],
                width=l if not type(l) == pd.Timedelta else l.total_seconds() * 1000,
                marker=dict(color="#cccccc"),
                text="{}".format(len(X[bins == cat]) if len(y) == 0 else y[i]),
                textposition='outside',
                name='',
                cliponaxis=False
            ),
            row=1,
            col=1
        )
    
    fig.add_trace(go.Scatter(
            x=X,
            y=np.ones(N),
            mode='markers',
            marker=dict(
                color="#586e75"
            ),
            text=text,
            name=""
        ),
        row=2,
        col=1,
    )
    
    fig.update_layout(
        
        xaxis=dict(
            showgrid=False
        ),
        yaxis=dict(
            showgrid=False,
            showticklabels=False
        ),
        yaxis2=dict(
            showgrid=False,
            showticklabels=False
        ),  
        showlegend=False,
        paper_bgcolor='rgba(0,0,0,0)',
        plot_bgcolor='rgba(0,0,0,0)',
        bargap=0.,
        font=dict(
            family="Arial",
            size=14,
            color="#586e75"
        )
    )
    
    return fig

In [ ]:
X = ccf.sort_values().reindex()
bps = v_optimal_dp(X, K=6)
bins = pd.cut(pd.Index(np.arange(len(X))), bps, include_lowest=True, right=False)
cats = bins.categories

fig = variable_width_distplot_factory(X, cats, bins, ccf.index)

fig.update_layout(
    title=dict(
        text="How delay correlates with time",
        xref="paper",
        x=0., y=1.
    ),
    annotations = [
        dict(
            xref='paper',
            yref='paper',
            x=0., y=1.2,
            showarrow=False,
            text ='This is a V-optimal histogram with 6 buckets for the correlation coefficient between delay (the time between CVE entries)' + 
            '<br>and time for the top-50 products. Lower values indicate that the number of exploits increases as time goes on;<br>' +
            'higher values indicate the opposite.',
            valign='top',
            align='left'
        )
    ]
)

fig.show()

# 2. Approximation 1: Exponentially-distributed data
For exponentially distributed data, we can use the approximation from 

In [ ]:
def approx_histogram_expon(a):
    # Segmentation approximation for exponentially distributed time series
    bp = [0,1]
    for i in range(2, len(a)):
        if np.mean(a[bp[-2]:bp[-1]]) < np.mean(a[bp[-1]:i]):
            while len(bp) > 1 and np.mean(a[bp[-2]:bp[-1]]) < np.mean(a[bp[-1]:i]):
                bp.pop()
        bp.append(i)
    bp.append(len(a))
    return np.array(bp)

In [ ]:
ps = ['android', 'internet_explorer', 'windows_xp']
for product, title in zip(ps, map(lambda x: ' '.join([s.title() if len(s) > 2 else s.upper() for s in x.split('_')]), ps)):
    # Delay
    p = pd.to_datetime(products[products.vulnerable_product == product].join(cve.pub_date).pub_date.sort_values().dropna())
    X = (p.iloc[1:].values - p.iloc[:-1].values).astype('timedelta64[m]')

    # Exponential segmentation
    bps = approx_histogram_expon(X)
    bins = pd.cut(pd.Index(np.arange(len(X))), bps, include_lowest=True, right=False)
    cats = bins.categories
    ys = np.array([np.mean(X[bins==cat]) for cat in cats])
    
    fig = variable_width_distplot_factory(p.iloc[:-1], cats, bins, p.index[:-1], np.round(ys.astype('float')/60/24, 2))
    fig.update_layout(
        title=title + " delay segmentation",
        yaxis=dict(type='log', showticklabels=True, title='Average Delay (in days)', dtick=1)
    )
    fig.show()